Following https://www.kaggle.com/code/awsaf49/sartorius-mmdetection-train

## Import

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import random
import pandas as pd
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torchvision
import torchvision.models as models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from tqdm.auto import tqdm
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

data_path = '/kaggle/input/global-wheat-detection/'

## Seed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    

seed_everything(41) # Seed 고정

## ConFig

In [4]:
CFG = {
    'NUM_CLASS':1,
    'IMG_SIZE':1024,
    'EPOCHS':10,
    'LR':1e-3 * 5,
    'BATCH_SIZE':16,
    'SEED':41
}

## Data

In [5]:
train_path = data_path +'train/'
test_path = data_path + 'test/'
df = pd.read_csv(data_path+'train.csv',index_col = False)
df_sub = pd.read_csv(data_path+'sample_submission.csv', index_col = False)
df_sub['image_path'] = data_path + 'test/' + df_sub['image_id'] + '.jpg'
df['image_path'] = data_path + 'train/' + df['image_id'] + '.jpg'
tmp_df = df.drop_duplicates(subset=["image_id", "image_path"]).reset_index(drop=True)
tmp_df["bbox"] = df.groupby("image_id")["bbox"].agg(list).reset_index(drop=True)
df = tmp_df.copy()
df['num_ins'] = df.bbox.map(lambda x: len(x))

## DataSet

In [6]:
def collate_fn(batch):
    images, targets_boxes, targets_labels = tuple(zip(*batch))
    images = torch.stack(images, 0)
    targets = []
    
    for i in range(len(targets_boxes)):
        target = {
            "boxes": targets_boxes[i],
            "labels": targets_labels[i]
        }
        targets.append(target)

    return images, targets

class CustomDataset(Dataset):
    def __init__(self, df, train=True,valid=True,transforms=None):
        self.data = df 
        self.transforms = transforms
        self.train = train
        
    def parse_boxes(self, box_data):
        boxes = []
        labels = []
        for value in box_data:
            value = eval(value)
            x_min, y_min = round(int(value[0])), round(int(value[1]))
            x_max, y_max = round(int(value[2])) + x_min, round(int(value[3]))+ y_min
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(1)

        return torch.tensor(boxes, dtype=torch.float32), torch.tensor(labels, dtype=torch.int64)

    def __getitem__(self, idx):
        img_path = self.data['image_path'][idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.train:
            box_data = self.data.iloc[idx,3]
            boxes, labels = self.parse_boxes(box_data)
            # Background = 0

            if self.transforms is not None:
                transformed = self.transforms(image=img, bboxes=boxes, labels=labels)
                img, boxes, labels = transformed["image"], transformed["bboxes"], transformed["labels"]
                
            return img, torch.tensor(boxes, dtype=torch.float32), torch.tensor(labels, dtype=torch.int64)

        else:
            if self.transforms is not None:
                transformed = self.transforms(image=img)
                img = transformed["image"]
            return img

    def __len__(self):
        return len(self.data)

In [7]:
transform = A.Compose([
        A.Flip(0.5),
        A.Normalize(),
        ToTensorV2(),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
transform_valid = A.Compose([
        A.Normalize(),
        ToTensorV2(),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
transform_test = A.Compose([
        A.Normalize(),
        ToTensorV2(),
    ])

In [8]:
train_df = df[:3000]
valid_df = df[3000:]
valid_df.reset_index(drop=True, inplace=True)
train_dataset = CustomDataset(train_df, train=True, transforms = transform)
valid_dataset = CustomDataset(valid_df, train=True, transforms = transform_valid)
test_dataset = CustomDataset(df_sub, train=False, transforms = transform_test)

train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, collate_fn=collate_fn, num_workers = 4)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn, num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

## Model

In [9]:
def build_model(num_classes=CFG['NUM_CLASS']+1):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

## Metric

In [10]:

def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area

def find_best_match(gts, pred, pred_idx, threshold = 0.5, form = 'pascal_voc', ious=None) -> int:
    """Returns the index of the 'best match' between the
    ground-truth boxes and the prediction. The 'best match'
    is the highest IoU. (0.0 IoUs are ignored).

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        pred: (List[Union[int, float]]) Coordinates of the predicted box
        pred_idx: (int) Index of the current predicted box
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (int) Index of the best match GT box (-1 if no match above threshold)
    """
    best_match_iou = -np.inf
    best_match_idx = -1

    for gt_idx in range(len(gts)):
        
        if gts[gt_idx][0] < 0:
            # Already matched GT-box
            continue
        
        iou = -1 if ious is None else ious[gt_idx][pred_idx]

        if iou < 0:
            iou = calculate_iou(gts[gt_idx], pred, form=form)
            
            if ious is not None:
                ious[gt_idx][pred_idx] = iou

        if iou < threshold:
            continue

        if iou > best_match_iou:
            best_match_iou = iou
            best_match_idx = gt_idx

    return best_match_idx
def calculate_precision(gts, preds, threshold = 0.5, form = 'coco', ious=None) -> float:
    """Calculates precision for GT - prediction pairs at one threshold.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (float) Precision
    """
    n = len(preds)
    tp = 0
    fp = 0
    
    # for pred_idx, pred in enumerate(preds_sorted):
    for pred_idx in range(n):

        best_match_gt_idx = find_best_match(gts, preds[pred_idx], pred_idx,
                                            threshold=threshold, form=form, ious=ious)

        if best_match_gt_idx >= 0:
            # True positive: The predicted box matches a gt box with an IoU above the threshold.
            tp += 1
            # Remove the matched GT box
            gts[best_match_gt_idx] = -1

        else:
            # No match
            # False positive: indicates a predicted box had no associated gt box.
            fp += 1

    # False negative: indicates a gt box had no associated predicted box.
    fn = (gts.sum(axis=1) > 0).sum()

    return tp / (tp + fp + fn)


def calculate_image_precision(gts, preds, thresholds = (0.5,), form = 'coco') -> float:
    """Calculates image precision.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        thresholds: (float) Different thresholds
        form: (str) Format of the coordinates

    Return:
        (float) Precision
    """
    n_threshold = len(thresholds)
    image_precision = 0.0
    
    ious = np.ones((len(gts), len(preds))) * -1
    # ious = None

    for threshold in thresholds:
        precision_at_threshold = calculate_precision(gts.copy(), preds, threshold=threshold,
                                                     form=form, ious=ious)
        image_precision += precision_at_threshold / n_threshold

    return image_precision

## Train

In [11]:
def train(model, train_loader, optimizer, scheduler, device):
    model.to(device)

    best_score = 0
    best_model = None
    use_amp = True
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for images, targets in tqdm(iter(train_loader)):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            optimizer.zero_grad()
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            train_loss.append(losses.item())
            
            losses.backward()
            optimizer.step()


        if scheduler is not None:
            scheduler.step()
        
        tr_loss = np.mean(train_loss)

        print(f'Epoch [{epoch}] Train loss : [{tr_loss:.5f}]\n')
        
            
        model.eval()
        validation_image_precisions = []
        for images, targets in tqdm(iter(valid_loader)):
            with torch.no_grad():
                images = [img.to(device) for img in images]
                outputs = model(images)
            for idx, output in enumerate(outputs):
                boxes = output["boxes"].cpu().numpy()
                target = targets[idx]
                boxes_target = target['boxes'].cpu().numpy()
                score = calculate_image_precision(boxes_target, boxes, thresholds = (0.50,0.55, 0.60, 0.65, 0.70, 0.75 ), form = 'pascal_voc' )
                validation_image_precisions.append(score)
        print(f'Epoch [{epoch}] ' +"Validation IOU: {0:.4f}".format(np.mean(validation_image_precisions)))
        if np.mean(validation_image_precisions) > best_score:
              bests_score = np.mean(validation_image_precisions)
              best_model = model
    return best_model

In [12]:
model = build_model()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=CFG['LR'], momentum=0.9, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

infer_model = train(model, train_loader, optimizer, scheduler, device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 218MB/s]


  0%|          | 0/188 [00:00<?, ?it/s]

Epoch [1] Train loss : [1.56371]



  0%|          | 0/373 [00:00<?, ?it/s]

Epoch [1] Validation IOU: 0.0103


...

Epoch [9] Validation IOU: 0.0142


  0%|          | 0/188 [00:00<?, ?it/s]

Epoch [10] Train loss : [1.54017]



  0%|          | 0/373 [00:00<?, ?it/s]

Epoch [10] Validation IOU: 0.0152


## Inference

In [13]:

model = infer_model
id = 0
for images in tqdm(iter(test_loader)):
    with torch.no_grad():
        images = [img.to(device) for img in images]
        outputs = model(images)
        for idx, output in enumerate(outputs):
            boxes = output["boxes"].cpu().numpy()
            scores = output["scores"].cpu().numpy()
            precision = ''
            for i, box in enumerate(boxes):
                xmin, ymin, xmax, ymax = (box[0]) * 1024 / CFG['IMG_SIZE'], box[1] * 1024 / CFG['IMG_SIZE'], box[2] * 1024 / CFG['IMG_SIZE'],box[3] * 1024 / CFG['IMG_SIZE']
                x, y, width, height = xmin, ymin, xmax-xmin, ymax-ymin
                one_precision = f'{scores[i]} {x} {y} {width} {height} '
                precision += one_precision
    df_sub.iloc[id, 1] = precision
    id += 1
df_sub.to_csv('result.csv', index = False)

                   

  0%|          | 0/10 [00:00<?, ?it/s]

## Metric

In [14]:
validation_image_precisions = []
for images, targets in tqdm(iter(valid_loader)):
    with torch.no_grad():
        images = [img.to(device) for img in images]
    for idx, output in enumerate(outputs):
        target = targets[idx]
        boxes_target = target['boxes'].cpu().numpy()
        boxes = boxes_target.copy()
        score = calculate_image_precision(boxes_target, boxes, thresholds = (0.50,0.55, 0.60, 0.65, 0.70, 0.75 ), form = 'pascal_voc' )
        
        validation_image_precisions.append(score)
    print(f'Epoch [1] ' +"Validation IOU: {0:.4f}".format(np.mean(validation_image_precisions)))

  0%|          | 0/373 [00:00<?, ?it/s]

Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] Validation IOU: 1.0000
Epoch [1] 